# Measurement

This notebook showcases how to use **pyecsca** to generate and measure an ECC implementation.
This example use the ChipWhisperer-Lite board, along with the UFO target board (with an `STM32F3` target on top)
and a PicoScope 5000 oscilloscope to measure.

## Oscilloscope setup

First we will setup the scope. Channel `A` will be used for the power signal, so we
connect the `MEASURE` SMA plug (on the UFO board) to the scope `A` input via an SMA-BNC cable. Channel
`B` will be used for the trigger, so we connect a probe to `TP2` point on the UFO board and connect it
to input `B` on the scope. 

![measurement setup](img/measurement_setup.jpg)

Next we connect to the scope and display its identifier.

In [ ]:
from pyecsca.sca.scope import PS5000Scope

scope = PS5000Scope()
scope.open()
print(scope.get_variant())

Then we setup the channels, `A` in AC coupling with 1 Volt range, `B` in DC coupling with 5 Volt range.

In [ ]:
scope.setup_channel(channel="A", coupling="AC", range=1.0, enable=True)
scope.setup_channel(channel="B", coupling="DC", range=5.0, enable=True)

Then we set the frequency and amount of samples. We set 4.6 MHz and 32M samples,
which should lead to a 7 second capture time (which should cover the long scalar multiplication operation on the chip ~ 6s).

In [ ]:
scope.setup_frequency(frequency=4_600_000, samples=32_000_000)

Next we setup the trigger on channel `B`. We also set channel `A` as the channel to capture.

In [ ]:
scope.setup_trigger(channel="B", threshold=1.0, direction="rising", delay=0, timeout=2000, enable=True)
scope.setup_capture(channel="A", enable=True)

## Device setup

The `STM32F3` UFO target board is used next, we now will generate build an ECC implementation.

![measurement ufo](img/measurement_ufo.jpg)

In [ ]:
import tempfile

from os.path import join
from pyecsca.codegen.common import Platform, DeviceConfiguration
from pyecsca.codegen.render import render_and_build
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.configuration import *

platform = Platform.STM32F3
hash_type = HashType.SHA1
mod_rand = RandomMod.REDUCE
mult = Multiplication.BASE
sqr = Squaring.BASE
red = Reduction.BASE

model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
add = coords.formulas["add-1998-cmo"]
dbl = coords.formulas["dbl-1998-cmo"]
scl = coords.formulas["z"]
formulas = [add, dbl, scl]
scalarmult = LTRMultiplier(add, dbl, scl)

config = DeviceConfiguration(model, coords, formulas, scalarmult, 
							 hash_type, mod_rand, mult, sqr, red,
							 platform, True, True, True)

tmpdir = tempfile.mkdtemp()
directory, elf_file, hex_file, res = render_and_build(config, tmpdir)
fw = join(tmpdir, hex_file)

Now we will create a target and flash the implementation on it.
The target constructor requires to know some parameters of the configuration,
to be able to communicate with it.

In [ ]:
from pyecsca.codegen.client import DeviceTarget

target = DeviceTarget(model=config.model, coords=config.coords, platform=config.platform, timeout=10000)
target.flash(fw)

## Measurement

We can now connect to the target, arm the scope and generate a keypair on the target while measuring it,
then collect the trace.

In [ ]:
from pyecsca.sca.trace import Trace
from pyecsca.ec.curves import get_params
params = get_params("secg", "secp128r1", "projective")

target.connect()
target.set_params(params)
scope.arm()
priv, pub = target.generate()
trace_data = scope.capture("A", 2)
trace = Trace("generate", None, samples=trace_data)
target.disconnect()

print(trace)

After all measurements are done, we disconnect from the scope.

In [ ]:
scope.close()